In [35]:
!pip install speechbrain==0.5.13

In [7]:
import speechbrain as sb
sb.__version__

'0.5.13'

In [8]:
print(dir(sb))

['Brain', 'Stage', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'alignment', 'core', 'create_experiment_directory', 'dataio', 'decoders', 'f', 'lm', 'lobes', 'nnet', 'os', 'parse_arguments', 'pretrained', 'processing', 'tokenizers', 'utils', 'version']


In [9]:
print('pretrained', dir(sb.pretrained))
print('\n')
print('dataio', dir(sb.dataio))
print('\n')
print('decoders', dir(sb.decoders))
print('\n')
print('processing', dir(sb.processing))
print('\n')
print('tokenizers', dir(sb.tokenizers))


pretrained ['AudioNormalizer', 'DDP', 'DP', 'DataPipeline', 'EncodeDecodePipelineMixin', 'EncoderASR', 'EncoderClassifier', 'EncoderDecoderASR', 'EndToEndSLU', 'F', 'GraphemeToPhoneme', 'HIFIGAN', 'PaddedBatch', 'PaddedData', 'Pretrained', 'SNREstimator', 'SepformerSeparation', 'SimpleNamespace', 'SpeakerRecognition', 'SpectralMaskEnhancement', 'SyncBatchNorm', 'Tacotron2', 'VAD', 'WaveformEnhancement', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'fetch', 'fetching', 'foreign_class', 'hashlib', 'import_from_path', 'interfaces', 'lengths_arg_exists', 'load_hyperpyyaml', 'logger', 'logging', 'run_on_main', 'sentencepiece', 'speechbrain', 'split_path', 'sys', 'torch', 'torchaudio']


dataio ['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'batch', 'dataio', 'dataloader', 'dataset', 'encoder', 'filename', 'iterators', 'legacy', 'os', 'pr

In [10]:
from speechbrain.dataio.dataio import read_audio
from IPython.display import Audio

In [11]:
#Download pretrained SpeakerRecognition from SpeechBrain
from speechbrain.pretrained import SpeakerRecognition
run_opts = {"device": "cuda"}
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", 
                                               savedir="../models/speechbrain/spkrec-ecapa-voxceleb", 
                                               run_opts=run_opts
                                               )


In [12]:
help(verification)

Help on SpeakerRecognition in module speechbrain.pretrained.interfaces object:

class SpeakerRecognition(EncoderClassifier)
 |  SpeakerRecognition(*args, **kwargs)
 |  
 |  A ready-to-use model for speaker recognition. It can be used to
 |  perform speaker verification with verify_batch().
 |  
 |  ```
 |  Example
 |  -------
 |  >>> import torchaudio
 |  >>> from speechbrain.pretrained import SpeakerRecognition
 |  >>> # Model is downloaded from the speechbrain HuggingFace repo
 |  >>> tmpdir = getfixture("tmpdir")
 |  >>> verification = SpeakerRecognition.from_hparams(
 |  ...     source="speechbrain/spkrec-ecapa-voxceleb",
 |  ...     savedir=tmpdir,
 |  ... )
 |  
 |  >>> # Perform verification
 |  >>> signal, fs = torchaudio.load("tests/samples/single-mic/example1.wav")
 |  >>> signal2, fs = torchaudio.load("tests/samples/single-mic/example2.flac")
 |  >>> score, prediction = verification.verify_batch(signal, signal2)
 |  
 |  Method resolution order:
 |      SpeakerRecognition
 |

In [13]:
score, prediction = verification.verify_files('../data/cs12 1m sample.ogg', '../data/cs12 sample2.ogg')
score, prediction

(tensor([0.5168], device='cuda:0'), tensor([True], device='cuda:0'))

In [14]:
score, prediction = verification.verify_files('../data/cs12 1m sample.ogg', '../data/crux sample2.ogg')
score, prediction

(tensor([0.2378], device='cuda:0'), tensor([False], device='cuda:0'))

In [15]:
score, prediction = verification.verify_files('../data/cs12 1m sample.ogg', '../data/jadepixie sample2.ogg')
score, prediction

(tensor([0.0619], device='cuda:0'), tensor([False], device='cuda:0'))

In [16]:
score, prediction = verification.verify_files('../data/cs12 1m sample.ogg', '../data/sam sample2.ogg')
score, prediction

(tensor([0.1766], device='cuda:0'), tensor([False], device='cuda:0'))

So we've managed to use Speechbrain to perform speaker verification across two different files.  This is a great first step, but in order to accomplish what I'm thinking I will need to feed it segments tested against a list of known voice samples.  In short, I'm less interested in asking "is this the same speaker" and more "do I know this speaker".

I should find a way to segment the audio file into tokens and test individual tokens.

In [17]:
print(dir(sb.pretrained))

['AudioNormalizer', 'DDP', 'DP', 'DataPipeline', 'EncodeDecodePipelineMixin', 'EncoderASR', 'EncoderClassifier', 'EncoderDecoderASR', 'EndToEndSLU', 'F', 'GraphemeToPhoneme', 'HIFIGAN', 'PaddedBatch', 'PaddedData', 'Pretrained', 'SNREstimator', 'SepformerSeparation', 'SimpleNamespace', 'SpeakerRecognition', 'SpectralMaskEnhancement', 'SyncBatchNorm', 'Tacotron2', 'VAD', 'WaveformEnhancement', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'fetch', 'fetching', 'foreign_class', 'hashlib', 'import_from_path', 'interfaces', 'lengths_arg_exists', 'load_hyperpyyaml', 'logger', 'logging', 'run_on_main', 'sentencepiece', 'speechbrain', 'split_path', 'sys', 'torch', 'torchaudio']


In [18]:
## Voice Activity Detection
from speechbrain.pretrained import VAD

In [19]:
print(help(VAD))

Help on class VAD in module speechbrain.pretrained.interfaces:

class VAD(Pretrained)
 |  VAD(*args, **kwargs)
 |  
 |  A ready-to-use class for Voice Activity Detection (VAD) using a
 |  pre-trained model.
 |  
 |  Example
 |  -------
 |  >>> import torchaudio
 |  >>> from speechbrain.pretrained import VAD
 |  >>> # Model is downloaded from the speechbrain HuggingFace repo
 |  >>> tmpdir = getfixture("tmpdir")
 |  >>> VAD = VAD.from_hparams(
 |  ...     source="speechbrain/vad-crdnn-libriparty",
 |  ...     savedir=tmpdir,
 |  ... )
 |  
 |  >>> # Perform VAD
 |  >>> boundaries = VAD.get_speech_segments("tests/samples/single-mic/example1.wav")
 |  
 |  Method resolution order:
 |      VAD
 |      Pretrained
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initializes internal Module state, shared by both nn.Module and ScriptModule.
 |  
 |  apply_threshold(self, vad_prob, activation_th=0.5, d

In [32]:
from speechbrain.pretrained import VAD
# Model is downloaded from the speechbrain HuggingFace repo
detection = VAD.from_hparams(
    source="speechbrain/vad-crdnn-libriparty",
    savedir="../models/speechbrain/vad-crdnn-libriparty",
    run_opts=run_opts
)

In [54]:
detection.hparams.sample_rate

16000

In [53]:
audio_file = '../data/db short intro.ogg'
detection.get_speech_segments(audio_file)

ValueError: The detected sample rate is different from that set in the hparam file

In [56]:
import torchaudio
from torchaudio.transforms import Resample
waveform, sample_rate = torchaudio.load(audio_file)
waveform, sample_rate

(tensor([[0.0017, 0.0024, 0.0029,  ..., 0.0000, 0.0000, 0.0000],
         [0.0017, 0.0024, 0.0029,  ..., 0.0000, 0.0000, 0.0000]]),
 48000)

: 

In [47]:
resampler = Resample(orig_freq=sample_rate, new_freq=16000)

In [50]:
resampled_waveform = resampler(waveform)

In [51]:
type(resampled_waveform)

torch.Tensor

In [52]:
VAD.upsample_VAD

['compute_features', 'mean_var_norm', 'model']